In [ ]:
# Original code by L.Costantini - reviewed with G.Gallerani
# Updated with axes and signal attributes in order to generate the correct graph with myHDF5
# Updated with the attributes "URL" and "version"
# Updated with parameter reliability 

from datetime import datetime, timezone
import h5py
import numpy as np
from pathlib import Path

input_file = 'insert path of the input file'
output_dir = 'insert path of the output directory'

In [ ]:
input_file = Path(input_file)
output_dir = Path(output_dir)

# Extract data from the input file
data = np.loadtxt('insert input file')
wavelenght = data[1:, 0]
delay_time = data[0, 1:]
signal = data[1:, 1:]

In [ ]:
now_datetime = datetime.now()

output_file = output_dir/Path(f"FTAS_{now_datetime.strftime("%Y%m%d_%H%M%S")}.nxs")
f = h5py.File(output_file, "w")
f.attrs['default'] = 'entry'

In [ ]:
# /entry
f.create_group("entry")
f['/entry'].attrs["NX_class"]= "NXentry"
f['/entry'].attrs["default"] = "data"

In [ ]:
# /entry/definition
f['/entry'].create_dataset('definition',data='NXoptical_spectroscopy')

In [ ]:
# /entry/version

f['/entry/definition'].attrs["version"] = 'v2024.02'
f['/entry/definition'].attrs["URL"] = 'https://github.com/FAIRmat-NFDI/nexus_definitions/blob/fd58c03d6c1be6469c2aff92ae7649fe5ad38a63/contributed_definitions/NXoptical_spectroscopy.nxdl.xml'

# /entry/experiment_description
f['/entry'].create_dataset('experiment_description',data='FEMTOSECOND TRANSIENT ABSORPTION SPECTROSCOPY')
f['/entry/experiment_description'].attrs["description"] = 'Technique'

# /entry/end_time
f['/entry'].create_dataset('end_time',data=now_datetime.astimezone().isoformat())

# /entry/title
f['/entry'].create_dataset('title',data=str(input_file.stem))

# /entry/experiment_type and sub_type
f['/entry'].create_dataset('experiment_type', data='Transmission Spectroscopy')
f['/entry'].create_dataset('experiment_sub_type', data='Pump-Probe')

# /entry/user - (labelled as not necessary in the Word draft)
f['/entry'].create_group("user")
f['/entry/user'].create_dataset('name', data = 'Mario Rossi')
f['/entry/user'].create_dataset('e-mail', data = 'mario.rossi@cnr.it')
f['/entry/user'].create_dataset('affiliation', data = 'CNR-ISM @ Rome - EuroFEL Support Lab') 

In [ ]:
# Introduction of the Instrument group
f['/entry'].create_group("instrument")
f['/entry/instrument'].attrs['NX_class'] = "NXinstrument"

# /entry/instrument/source
f['/entry/instrument'].create_group("source_TYPE")
f['/entry/instrument/source_TYPE'].attrs["NX_class"] = "NXsource"
f['/entry/instrument/source_TYPE'].create_dataset('source', data = 'pump')
f['/entry/instrument/source_TYPE'].create_dataset('type', data = 'laser')
f['/entry/instrument/source_TYPE'].create_dataset('pulse_energy', data = '100')
f['/entry/instrument/source_TYPE/pulse_energy'].attrs['units']='nJ'

# /entry/instrument/beam/pump
f['/entry/instrument'].create_group("beam_pump")
f['/entry/instrument/beam_pump'].attrs['NX_class'] = "NXbeam"
f['/entry/instrument/beam_pump'].create_dataset('beam_type', data = 'pump')
f['/entry/instrument/beam_pump'].create_dataset('incident_wavelenght', data = '450')
f['/entry/instrument/beam_pump/incident_wavelenght'].attrs['units'] = 'nm'
f['/entry/instrument/beam_pump'].create_dataset('parameter_reliability', data = 'nominal')
f['/entry/instrument/beam_pump'].create_dataset('beam_polarization_type', data = 'linear')

# /entry/instrument/beam/probe
f['/entry/instrument'].create_group("beam_probe")
f['/entry/instrument/beam_probe'].attrs['NX_class'] = "NXbeam"
f['/entry/instrument/beam_probe'].create_dataset('beam_type', data = 'probe')
f['/entry/instrument/beam_probe'].create_dataset('incident_wavelenght', data = '350-750')
f['/entry/instrument/beam_probe/incident_wavelenght'].attrs['units'] = 'nm'
f['/entry/instrument/beam_probe'].create_dataset("parameter_reliability",data = "measured")
f['/entry/instrument/beam_probe'].create_dataset('beam_polarization_type', data = 'linear')

# /entry/instrument/detector
f['/entry/instrument'].create_group("detector_TYPE")
f['/entry/instrument/detector_TYPE'].attrs['NXclass'] = "NXdetector"
f['/entry/instrument/detector_TYPE'].create_dataset('detector_type', data = 'CCD')
f['/entry/instrument/detector_TYPE'].create_dataset('detector_channel_type', data = 'multichannel')
f['/entry/instrument/detector_TYPE'].create_dataset('count_time', data = '5')

In [ ]:
# /entry/data
f['/entry'].create_group("data")
f['/entry/data'].attrs['NX_class']= "NXdata"
f['/entry/data'].create_dataset("intensity", data = signal)
f['/entry/data'].attrs["signal"] = "intensity" 
f['/entry/data/intensity'].attrs["units"] = "mOD"
f['/entry/data'].create_dataset("wavelenght", data = wavelenght)
f['/entry/data/wavelenght'].attrs["units"] = "nm"
f['/entry/data'].attrs["axes"]=["wavelenght", "delay_time"] 
f['/entry/data'].create_dataset("delay_time", data = delay_time)
f['/entry/data/delay_time'].attrs["units"] = "ps"

# /entry/raw_data
f['/entry'].create_group("raw_data")
f['/entry/raw_data'].attrs['NX_class']= "NXdata"
f['/entry/raw_data'].attrs['signal']= "raw"
f['/entry/raw_data'].create_dataset("raw", data = data) 
#Contains the raw data collected by the detector before calibration. The data which is considered raw might change from experiment to experiment due to hardware pre-processing of the data. This field ideally collects the data with the lowest level of processing possible.

# /entry/sample

f['/entry'].create_group("sample")
f['/entry/sample'].attrs['NXclass']= "NXsample"
f['/entry/sample'].create_dataset('sample_name', data = 'formamidinium lead bromide')
f['/entry/sample'].create_dataset('chemical_formula', data = 'FAPbBr3')
f['/entry/sample'].create_dataset('physical_form', data ='solid')

In [ ]:
f.close()

In [ ]:
# Visualize the NeXus file tree
from nexusformat.nexus import *

test=nxload(output_file)
print(test.tree)